In [12]:
#import necessary packages
import pandas as pd 
import splitfolders
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers import scikit_learn
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
import joblib

In [2]:
def pickle_model(model):
    
    file_name = input("What would you like to name the model?")
    file_name = file_name.replace(' ', '')
    # use the built-in open() function to open a file
    output_file = open(f"./models/{file_name}.pkl", "wb") # "wb" means "write as bytes"
    # dump the variable's contents into the file
    joblib.dump(model, output_file)
    # close the file, ensuring nothing stays in the buffer
    output_file.close()
    

In [2]:
#use the splitfolders package to split our image data into train, validation, and test sets.
#this is now commented out so that we don't create a new folder each time we run the notebook

#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

In [10]:
simple_train_data = train_data.reshape(train_data.shape[0], -1)
simple_test_data = test_data.reshape(test_data.shape[0], -1)
simple_val_data = val_data.reshape(val_data.shape[0], -1)

In [11]:
train_data.shape

(3749, 150, 150, 1)

In [12]:
simple_model = models.Sequential()

In [13]:
simple_model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
simple_model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
simple_model.compile(optimizer='SGD', 
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [15]:
simple_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                270012    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 270,025
Trainable params: 270,025
Non-trainable params: 0
_________________________________________________________________


In [16]:
simple_fit = simple_model.fit(simple_train_data,
                                  train_labels,
                                  epochs=5,
                                  batch_size=32,
                                  validation_data=(val_data,val_labels),
                                  verbose=1)

Epoch 1/5


ValueError: in user code:

    C:\Users\YueYu\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\YueYu\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\YueYu\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\YueYu\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\YueYu\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\YueYu\anaconda3\envs

In [19]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150,150,1)))
cnn_model.add(layers.MaxPooling2D((2,2)))
cnn_model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,1)))       
cnn_model.add(layers.MaxPooling2D((2,2)))
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(12, activation='relu'))
cnn_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
cnn_model.summary()

In [20]:
cnn_model.compile(optimizer='SGD',
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [21]:
cnn_model_fit = cnn_model.fit(train_data, train_labels, epochs=10, 
                                    batch_size=32, validation_data= (val_data, val_labels), verbose=1)

Epoch 1/10
118/118 [==============================] - 77s 655ms/step - loss: 0.5617 - accuracy: 0.7263 - precision_1: 0.7302 - recall_1: 0.9912 - val_loss: 0.4950 - val_accuracy: 0.7297 - val_precision_1: 0.7297 - val_recall_1: 1.0000
Epoch 2/10
118/118 [==============================] - 75s 637ms/step - loss: 0.4245 - accuracy: 0.8050 - precision_1: 0.8165 - recall_1: 0.9452 - val_loss: 1.0808 - val_accuracy: 0.3280 - val_precision_1: 1.0000 - val_recall_1: 0.0791
Epoch 3/10
118/118 [==============================] - 72s 613ms/step - loss: 0.3358 - accuracy: 0.8680 - precision_1: 0.8892 - recall_1: 0.9357 - val_loss: 0.3619 - val_accuracy: 0.8269 - val_precision_1: 0.8127 - val_recall_1: 0.9912
Epoch 4/10
118/118 [==============================] - 74s 630ms/step - loss: 0.2730 - accuracy: 0.8821 - precision_1: 0.9085 - recall_1: 0.9324 - val_loss: 0.2604 - val_accuracy: 0.8889 - val_precision_1: 0.8774 - val_recall_1: 0.9854
Epoch 5/10
118/118 [==============================] - 72s 60

## Image Augmentation

In [3]:
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=30, 
                             width_shift_range=0.25, 
                             height_shift_range=0.25, 
                             shear_range=0.25, 
                             zoom_range=0.25, 
                             horizontal_flip=True,
                             vertical_flip=True)

In [4]:
train_generator = datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=4280,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary', 
        color_mode='grayscale')

Found 3749 images belonging to 2 classes.


In [5]:
#create a validation_generator from the training directory that has been created from the split folders package
validation_generator = datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=978,
                                                        class_mode='binary',
                                                        color_mode='grayscale')

Found 936 images belonging to 2 classes.


In [6]:
#create a test_generator from the training directory that has been created from the split folders package
test_generator = datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=1173,
                                                  class_mode='binary',
                                                  color_mode='grayscale')

Found 1107 images belonging to 2 classes.


In [7]:
train_data, train_labels = next(train_generator)
val_data, val_labels = next(validation_generator)
test_data, test_labels = next(test_generator)

## Simple Model

In [42]:
model = Sequential()
model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])


In [42]:
keras_model = KerasClassifier(model, epochs=10, verbose=2, validation_data=(val_data, val_labels))

In [43]:
simple_train_data = train_data.reshape(train_data.shape[0], -1)
simple_test_data = test_data.reshape(test_data.shape[0], -1)
simple_val_data = val_data.reshape(val_data.shape[0], -1)

In [44]:
simpel_results = model.fit(simple_train_data, train_labels, batch_size=32, epochs=10, validation_data=(val_data, val_labels))

TypeError: object of type 'NoneType' has no len()

In [30]:
pickle_model(simpel_results)

What would you like to name the model?simple model
INFO:tensorflow:Assets written to: ram://10b2da24-b5f2-4879-af78-59c456ec183b/assets


In [47]:
simpel_results.history

{'loss': [0.6107282042503357],
 'accuracy': [0.7135236263275146],
 'precision_3': [0.7480596899986267],
 'recall_3': [0.9159356951713562]}

In [31]:
train_loss = simpel_results.history['loss']
train_acc = simpel_results.history['accuracy']
val_loss = simpel_results.history['val_loss']
val_acc = simpel_results.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=simpel_results.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=simpel_results.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=simpel_results.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=simpel_results.epoch, y=val_acc, ax=ax2, label='val_accuracy');

KeyError: 'val_loss'

## CNN model

In [36]:
loss = 'binary_crossentropy'
metric_ls = ['accuracy', metrics.Precision(), metrics.Recall()]
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', patience=20, min_delta=0.01, baseline = 0.01, restore_best_weights=True)

In [37]:
def cnn_model():  
    model = Sequential()
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(32, (3,3), activation='relu'))       
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(12, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='Adam', loss=loss, metrics=metric_ls)
    return model

In [38]:
model = KerasClassifier(cnn_model, epochs=10, verbose=2)

In [39]:
params = dict(batch_size=[16, 32, 64])
              #'dropout_rate':[0.1, 0.2, 0.3, 0.4, 0.5]

In [40]:
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=-1)

In [41]:
results = grid.fit(train_data, train_labels, callbacks=[early_stop])

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\YueYu\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\YueYu\anaconda3\envs\tf\lib\site-packages\scikeras\wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\YueYu\anaconda3\envs\tf\lib\site-packages\scikeras\wrappers.py", line 762, in fit
    self._fit(
  File "C:\Users\YueYu\anaconda3\envs\tf\lib\site-packages\scikeras\wrappers.py", line 929, in _fit
    self._check_model_compatibility(y)
  File "C:\Users\YueYu\anaconda3\envs\tf\lib\site-p

In [95]:
results.best_score_

0.9466524243354797

In [89]:
train_loss = model.fit.history['loss']
train_acc = model.fit.history['accuracy']
val_loss = model.fit.history['val_loss']
val_acc = model.fit.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=results.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=grid.fit(train_data, train_labels).epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=grid.fit(train_data, train_labels).epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=grid.fit(train_data, train_labels).epoch, y=val_acc, ax=ax2, label='val_accuracy');

NameError: name 'model' is not defined